# This Note book follow Machine Learning Tutorial by Sentdex

In [81]:
import pandas as pd 
import quandl, math, datetime
import numpy as np
from sklearn import preprocessing, model_selection, svm
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from matplotlib import style

Quick retrive Google Stock price 

In [67]:
df = quandl.get('WIKI/GOOGL')

In [68]:
df.tail()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2018-03-21,1092.57,1108.70,1087.21,1094.00,1990515.0,0.0,1.0,1092.57,1108.70,1087.21,1094.00,1990515.0
2018-03-22,1080.01,1083.92,1049.64,1053.15,3418154.0,0.0,1.0,1080.01,1083.92,1049.64,1053.15,3418154.0
2018-03-23,1051.37,1066.78,1024.87,1026.55,2413517.0,0.0,1.0,1051.37,1066.78,1024.87,1026.55,2413517.0
2018-03-26,1050.60,1059.27,1010.58,1054.09,3272409.0,0.0,1.0,1050.60,1059.27,1010.58,1054.09,3272409.0
2018-03-27,1063.90,1064.54,997.62,1006.94,2940957.0,0.0,1.0,1063.90,1064.54,997.62,1006.94,2940957.0


Generate High-Close %Chg and %Chg Return

In [69]:
df['HL_PCT'] = (df['Adj. High'] / df['Adj. Close'] - 1) * 100
df['PCT_change'] = (df['Adj. Close'] / df['Adj. Open'] - 1) * 100

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3424 entries, 2004-08-19 to 2018-03-27
Data columns (total 14 columns):
Open           3424 non-null float64
High           3424 non-null float64
Low            3424 non-null float64
Close          3424 non-null float64
Volume         3424 non-null float64
Ex-Dividend    3424 non-null float64
Split Ratio    3424 non-null float64
Adj. Open      3424 non-null float64
Adj. High      3424 non-null float64
Adj. Low       3424 non-null float64
Adj. Close     3424 non-null float64
Adj. Volume    3424 non-null float64
HL_PCT         3424 non-null float64
PCT_change     3424 non-null float64
dtypes: float64(14)
memory usage: 401.2 KB


In [65]:
# Test shift() and dropna function
df['test'] = df['Adj. Close'].shift(-5)
df.dropna(inplace=True)
#df.tail(20)
df.tail(20)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3354 entries, 2004-08-19 to 2017-12-13
Data columns (total 16 columns):
Open           3354 non-null float64
High           3354 non-null float64
Low            3354 non-null float64
Close          3354 non-null float64
Volume         3354 non-null float64
Ex-Dividend    3354 non-null float64
Split Ratio    3354 non-null float64
Adj. Open      3354 non-null float64
Adj. High      3354 non-null float64
Adj. Low       3354 non-null float64
Adj. Close     3354 non-null float64
Adj. Volume    3354 non-null float64
HL_PCT         3354 non-null float64
PCT_change     3354 non-null float64
label          3354 non-null float64
test           3354 non-null float64
dtypes: float64(16)
memory usage: 445.5 KB


Assign forecast column  

In [71]:
forecast_col = 'Adj. Close'
df.fillna(-9999, inplace=True)

forecast_out = int(math.ceil(0.01*len(df))) #return number of forecast % items

df['label'] = df[forecast_col].shift(-forecast_out) # shift data up = forecast_out

df.dropna(inplace=True)


In [72]:
df.info()
df.tail()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3389 entries, 2004-08-19 to 2018-02-05
Data columns (total 15 columns):
Open           3389 non-null float64
High           3389 non-null float64
Low            3389 non-null float64
Close          3389 non-null float64
Volume         3389 non-null float64
Ex-Dividend    3389 non-null float64
Split Ratio    3389 non-null float64
Adj. Open      3389 non-null float64
Adj. High      3389 non-null float64
Adj. Low       3389 non-null float64
Adj. Close     3389 non-null float64
Adj. Volume    3389 non-null float64
HL_PCT         3389 non-null float64
PCT_change     3389 non-null float64
label          3389 non-null float64
dtypes: float64(15)
memory usage: 423.6 KB


,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,HL_PCT,PCT_change,label
Date,,,,,,,,,,,,,,,
2018-01-30,1177.72,1187.93,1174.51,1177.37,1792602.0,0.0,1.0,1177.72,1187.93,1174.51,1177.37,1792602.0,0.896914,-0.029718,1094.00
2018-01-31,1183.81,1186.32,1172.10,1182.22,1643877.0,0.0,1.0,1183.81,1186.32,1172.10,1182.22,1643877.0,0.346805,-0.134312,1053.15
2018-02-01,1175.99,1187.45,1169.36,1181.59,2774967.0,0.0,1.0,1175.99,1187.45,1169.36,1181.59,2774967.0,0.495942,0.476195,1026.55
2018-02-02,1127.42,1131.30,1111.17,1119.20,5798880.0,0.0,1.0,1127.42,1131.30,1111.17,1119.20,5798880.0,1.081129,-0.729098,1054.09
2018-02-05,1100.61,1114.99,1056.74,1068.76,3742469.0,0.0,1.0,1100.61,1114.99,1056.74,1068.76,3742469.0,4.325574,-2.893850,1006.94


Define Variable 

In [79]:
# X = np.array(df.drop(['label']))
# y = np.array(df[label])

# Normalize data 
X = np.array(df.drop(['label'], 1)) #Retrive all column except label
y = np.array(df['label'])
X = preprocessing.scale(X)
X = X[:-forecast_out]


array([ 1.10061000e+03,  1.11499000e+03,  1.05674000e+03,  1.06876000e+03,
        3.74246900e+06,  0.00000000e+00,  1.00000000e+00,  1.10061000e+03,
        1.11499000e+03,  1.05674000e+03,  1.06876000e+03,  3.74246900e+06,
        4.32557356e+00, -2.89384977e+00])